# Seminovos Fleet Processing Notebook

This notebook lets you transform raw CSV files from Localiza & Movida plus FIPE reference data into clean, analysis‑ready tables. It reads files already saved under the `raw/` and `data/` folders.

You can run each section (Shift+Enter) from top to bottom or Run All. If you update input CSVs later, you only need to re‑run from the “Path Configuration” section downward.

---
**High‑Level Output Files (all saved under `data/`):**
- `data/localiza/localiza_with_fipe_match_YYYYMMDD.csv` – Localiza rows + matched FIPE code + score
- `data/tuples/fipe_tuples_YYYYMMDD.csv` – Unique (FIPE code, model year) combinations observed
- `data/tables/localiza_table_YYYYMMDD.csv` – Localiza vendor table enriched with FIPE price
- `data/tables/movida_table_YYYYMMDD.csv` – Movida vendor table enriched with FIPE price
- `data/tables/fipe_table_YYYYMMDD.csv` – FIPE monthly price time series

If something fails, read the Troubleshooting section near the end.

## Glossary
- Normalization: Cleaning text so similar things look the same (e.g. accents removed, abbreviations expanded).
- Token: A single word after cleaning. We compare sets of tokens to measure similarity.
- FIPE code: Official identifier for a vehicle model/year in the FIPE price table.
- Match score: Number from 0 to ~1 estimating how good the FIPE match is (higher is better).
- Threshold: Minimum score we trust automatically.
- Premium vs FIPE: (Vendor asking price – FIPE reference price) / FIPE reference price.
- Tuple: A pair (FIPE code, model year) representing a unique market unit.
- Cache: Saved historical matches so we avoid recomputing the same work.

## Quick Start
1. Place new raw CSVs under `raw/localiza/` and `raw/movida/` (naming pattern already used in folder).
2. Ensure `data/fipe_models.csv` exists (mandatory) and a FIPE dump in `data/fipe/`.
3. Run each cell from top to bottom (Run All works too).
4. Grab exported tables from `data/tables/`.

If you only updated one vendor file, you still run everything – it will be fast because matching cache skips previously seen versions.

In [ ]:
# 1. Setup & Imports
# (You just run this cell – it prepares libraries and logging.)
import pandas as pd, numpy as np, re, json, logging, math
from pathlib import Path
from datetime import date, datetime
from difflib import SequenceMatcher
import unicodedata

# Display tweaks: wider tables in notebook
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 140)

# Simple logger so later steps show progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)7s | %(message)s')
log = logging.getLogger('offline')

## 2. Path Configuration & Input File Selection
Detect where the input CSVs live and pick the most recent ones automatically.

Folders used (created if missing):
- `raw/localiza/` – historical Localiza CSV exports (naming: `localiza_seminovos_YYYYMMDD.csv`)
- `raw/movida/` – historical Movida CSV exports (naming: `movida_seminovos_YYYYMMDD.csv`)
- `data/` – working data (e.g. `fipe_models.csv`)
- `data/fipe/` – FIPE price dump snapshots (`fipe_dump_YYYYMMDD.csv`)

What this section does:
1. Creates folders if they do not exist.
2. Picks the *latest* file matching each pattern based on modification time.
3. Logs which files will be used.

If a file you want is NOT selected (e.g. you added a newer one but want an older snapshot), manually set the variables after the cell runs, e.g.:
```
localiza_csv = Path('raw/localiza/localiza_seminovos_20250824.csv')
```
Then re-run from the next section onward.

In [ ]:
# Auto-detect helper
# Picks the most recently modified file matching "pattern" inside a folder.
def latest(pattern: str, base: Path) -> Path | None:
    files = sorted(base.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    return files[0] if files else None

# Define core directories (relative to this notebook location)
BASE = Path.cwd().parent / 'Seminovos'
DATA_DIR = BASE / 'data'
RAW_DIR = BASE / 'raw'
RAW_LOCALIZA = RAW_DIR / 'localiza'
RAW_MOVIDA = RAW_DIR / 'movida'
FIPE_DIR = DATA_DIR / 'fipe'
TUPLES_DIR = DATA_DIR / 'tuples'
TABLES_DIR = DATA_DIR / 'tables'
VERSION_MATCH_TABLE = DATA_DIR / 'localiza_version_match.csv'
MATCH_DIR = DATA_DIR / 'localiza'

for d in [DATA_DIR, RAW_LOCALIZA, RAW_MOVIDA, FIPE_DIR, TUPLES_DIR, TABLES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Auto-pick newest snapshots
# If you need to override manually for picking an older snapshot, set e.g.:
# localiza_csv = Path('raw/localiza/localiza_seminovos_20250824.csv')
localiza_csv = latest('localiza_seminovos_*.csv', RAW_LOCALIZA)
movida_csv = latest('movida_seminovos_*.csv', RAW_MOVIDA)
# FIPE models master (must exist)
fipe_models_csv = (DATA_DIR / 'fipe_models.csv') if (DATA_DIR / 'fipe_models.csv').exists() else Path('fipe_models.csv')
if not fipe_models_csv.exists(): fipe_models_csv = None
# FIPE dump with pricing
fipe_dump_csv = latest('fipe_dump_*.csv', FIPE_DIR)

log.info('Localiza CSV: %s', localiza_csv)
log.info('Movida   CSV: %s', movida_csv)
log.info('FIPE models CSV: %s', fipe_models_csv)
log.info('FIPE dump CSV: %s', fipe_dump_csv)

## 3. Utility Functions (Dates, Logging, Helpers)
Small helper functions used later:
- Date stamps for file names (so outputs never overwrite previous days)
- Safe directory creation
- Price cleaning: turns messy price strings ("R$ 58.900,00") into an integer number of Reais (58900)

You normally do **not** need to edit anything here.

In [ ]:
# --- Utility helpers ---

def ymd_compact(d: date | None = None) -> str:
    """Return YYYYMMDD string for filenames (defaults to today)."""
    return (d or date.today()).strftime('%Y%m%d')

def today_iso() -> str:
    """Return ISO date YYYY-MM-DD for logging / columns."""
    return date.today().isoformat()

def ensure_dir(p: Path):
    """Create directory (and parents) if absent (no error if it exists)."""
    p.mkdir(parents=True, exist_ok=True)

def clean_price_to_int(x):
    """Normalize a price value (string like 'R$ 58.900,00' or number) to an int or <NA>."""
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return pd.NA
    if isinstance(x, (int, np.integer)):
        return int(x)
    if isinstance(x, (float, np.floating)):
        return int(round(float(x)))
    # Remove currency symbols & thousand separators, unify decimal marker
    s = re.sub(r'[^\d\.,]', '', str(x))
    if ',' in s and '.' in s:
        s = s.replace('.', '').replace(',', '.')  # "58.900,00" -> "58900.00"
    elif ',' in s:
        s = s.replace(',', '.')
    try:
        return int(round(float(s)))
    except Exception:
        return pd.NA

## 4. Normalization Helpers (Accents, Text Cleaning, Tokens)
Why we normalize:
Marketplace data has many variations (accents, punctuation, abbreviations: e.g. "Hig" vs "Highline"). To reliably match vehicles to FIPE we convert text into a *canonical* form.

Key concepts:
- strip_accents → "caminhão" becomes "caminhao"
- norm_text → heavy cleaning + standardizing abbreviations ("aut", "mec", model trims, etc.)
- generic_norm_text → lighter cleaning when we only need broad grouping
- norm_brand → groups aliases to a single brand ("gm", "chevrolet")
- tokset → converts normalized string into a set of unique words (tokens) for similarity scoring
- score_best_fipe_for_key → computes the best FIPE model candidate with a relevance score

Unless you add new brand aliases or trim patterns, leave this section unchanged.

In [ ]:
# Canonical normalization & helper functions (from fleet.py)
# These functions standardize messy vendor text so we can compare with FIPE data.
import re, unicodedata, pandas as pd
from difflib import SequenceMatcher
import numpy as np

# ---------------- Text Normalization Core ----------------

def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFD", str(s))
        if unicodedata.category(c) != "Mn"
    )

def remove_duplicate_words(text: str) -> str:
    # Keep first occurrence of each word (case-insensitive) to reduce noise
    seen = set(); result=[]
    for word in text.split():
        key = word.casefold()
        if key not in seen:
            seen.add(key); result.append(word)
    return " ".join(result)

def norm_text(s: str) -> str:
    """Aggressive normalization of version names (trims, engines, special editions)."""
    import pandas as pd, re
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s0 = strip_accents(str(s).lower())
    # Standardize punctuation / separators
    s0 = s0.replace(",", ".")
    s0 = s0.replace("c/ar", "").replace("c/ ar","")
    s0 = re.sub(r'/', ' ', s0)
    # Replace abbreviations & partial forms
    s0 = re.sub(r'(?i)(?<!\w)(?:T\.)(?!\w)', 'turbo', s0)
    s0 = re.sub(r"[^a-z0-9\.\s]", " ", s0)
    s0 = re.sub(r"\bautomatic[oa]\b|\bat\b|\baut(?:\.|o)?\b", "aut", s0)
    s0 = re.sub(r"\bman(?:ual)?\b|\bmecanico\b", "mec", s0)
    s0 = re.sub(r"\bt\s?si\b", "tsi", s0)
    s0 = re.sub(r"\b(\d{2,4}(?:i|d))\s*a\b", r"\1 aut", s0)
    # Remove weird dots inside words
    s0 = re.sub(r'(?<=[A-Za-z])\.(?=[A-Za-z])', '. ', s0)
    s0 = re.sub(r'(?<=[A-Za-z])\.', '', s0)
    # Normalize common marketing trim abbreviations
    s0 = re.sub(r'(?i)(?<!\w)(?:perf|perfor|performa|performance|p)(?!\w)', 'performance', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:long)(?!\w)', 'longitude', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:sportb|SPB|SB)(?!\w)', 'sportback', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:prest)(?!\w)', 'prestige', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:ultim)(?!\w)', 'ultimate', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:insc)(?!\w)', 'inscription', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:xdrive30e)(?!\w)', 'xdrive 30e', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:cp)(?!\w)', 'cs plus', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:7l)(?!\w)', '', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:hurric|hurr)(?!\w)', 'hurricane', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:overl)(?!\w)', 'overland', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:dies|die)(?!\w)', 'diesel', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:tb)(?!\w)', 'turbo', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:sed)(?!\w)', 'sedan', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:step)(?!\w)', 'stepway', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:hig)(?!\w)', 'highline', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:limit)(?!\w)', 'limited', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:plat)(?!\w)', 'platinum', s0)
    s0 = re.sub(r'(?i)\b\d+[pv]\b', '', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:exclu)(?!\w)', 'exclusive', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:t270)(?!\w)', 'turbo 270', s0)
    s0 = re.sub(r'(?i)(?<!\w)(?:comfort|comfor)(?!\w)', 'comfortline', s0)
    # Specific mapping corrections
    s0 = re.sub(r'(?i)\bONIX\s+HATCH\s+PREM\.\b', 'onix hatch premier', s0)
    s0 = re.sub(r'(?i)\bONIX\s+SEDAN\s+PREM\.\b', 'onix sedan premier', s0)
    s0 = re.sub(r'(?i)\bONIX\s+SEDAN\s+Plus+\s+PREM\.\b', 'onix sedan plus premier', s0)
    s0 = re.sub(r'(?i)\bONIX\s+SD\.\s+P\.\s+PR\.\b', 'onix sedan plus premier', s0)
    s0 = re.sub(r'(?i)\bFastback\s+Limited+\s+Ed\.\b', 'fastback limited edition', s0)
    s0 = re.sub(r'(?i)\bAIRCROSS\s+F\.\b', 'aircross feel', s0)
    # Put space in xc60, xc90 etc. if needed for uniform tokenization
    s0 = re.sub(r'(?<=xc)(\d+)', r' \1', s0)
    # Remove stray marketing "new" unless part of specific model names
    s0 = re.sub(r'\bnew\b(?![\s-]*(?:range|beetle)\b)', '', s0, flags=re.IGNORECASE)
    s0 = remove_duplicate_words(s0)
    s0 = re.sub(r"\s+", " ", s0).strip()
    return s0

def generic_norm_text(s: str) -> str:
    """Lighter normalization: keep only alphanumerics + collapse spaces."""
    import re
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s0 = strip_accents(str(s).lower())
    s0 = re.sub(r"[^a-z0-9\s]", " ", s0)
    s0 = re.sub(r'(?<=xc)(\d+)', r' \1', s0)
    s0 = re.sub(r"\s+", " ", s0).strip()
    return s0

def norm_brand(s: str) -> str:
    """Brand alias consolidation (add here if new alias surfaces)."""
    s0 = generic_norm_text(s)
    aliases = {
        "vw - volkswagen":"volkswagen","vw volkswagen":"volkswagen","volks":"volkswagen","volkswagem":"volkswagen",
        "gm":"chevrolet","gm - chevrolet":"chevrolet","gm chevrolet":"chevrolet","chevy":"chevrolet",
        "mercedes-benz":"mercedes benz","mb":"mercedes benz","caoa chery":"chery","caoa chery/chery":"chery","great wall":"gwm"
    }
    return aliases.get(s0, s0)

def tokset(s: str) -> set:
    return set(generic_norm_text(s).split())

def extract_engine(s: str):
    m = re.search(r"\b(\d\.\d)\b", str(s))
    return m.group(1) if m else None

# ---------------------------- Type normalization  -----------------------------

def normalize_type(raw_type, fuel_sigla):
    if fuel_sigla:
        fs = str(fuel_sigla).upper().strip()
        if fs in {'E','H'}:
            return 'ev'
    if raw_type is None or (isinstance(raw_type, float) and pd.isna(raw_type)):
        return ""
    s0 = strip_accents(str(raw_type)).upper().strip()
    mapping = {
        'SEDAN':'SEDAN','SEDÃ':'SEDAN','SEDA':'SEDAN',
        'HATCH':'HATCH','HATCHBACK':'HATCH',
        'SUV':'SUV',
        'PICAPE':'PICKUP/VANS','PICAPE CABINE DUPLA':'PICKUP/VANS','PICK-UP':'PICKUP/VANS','PICKUP':'PICKUP/VANS','CAMINHONETE':'PICKUP/VANS','CABINE SIMPLES':'PICKUP/VANS',
        'UTILITÁRIO':'PICKUP/VANS', 'UTILITARIO':'PICKUP/VANS','FURGAO':'PICKUP/VANS','MINIVAN':'PICKUP/VANS','VAN':'PICKUP/VANS','CARGA':'PICKUP/VANS',
        'ELETRICO':'EV','HIBRIDO':'EV',
        'COUPE':'PREMIUM','SPORTBACK':'PREMIUM','GRAN COUPE':'PREMIUM','FASTBACK':'PREMIUM',
        'PARTICULAR':'OTHER','OUTROS':'OTHER','OUTRO':'OTHER'
    }
    return mapping.get(s0, s0).lower()

# ---------------- Matching scorer (simplified canonical subset) ----------------

def score_best_fipe_for_key(brand_norm: str, model_norm: str, version_norm: str, model_year: int, fipe_df: pd.DataFrame, threshold: float):
    """Return best (Model, FIPE code, Year, Score, status) for provided normalized key."""
    if not version_norm or not model_norm or pd.isna(model_year):
        return (None, None, None, 0.0, "unmatched")
    m_tokens = [re.escape(t) for t in model_norm.split() if t]
    if not m_tokens:
        return (None, None, None, 0.0, "unmatched")
    token_pattern = "(?=.*" + ")(?=.*".join(m_tokens) + ")"
    cand_mask = (
        (fipe_df["_brand_norm"] == brand_norm) &
        (fipe_df["AnoModelo"] == int(model_year)) &
        fipe_df["_model_norm"].str.contains(token_pattern, regex=True, na=False)
    )
    cand = fipe_df[cand_mask]
    if cand.empty:
        return (None, None, None, 0.0, "unmatched")
    v_toks = tokset(version_norm)
    v_engine = extract_engine(version_norm)
    s_best=-1.0; m_best=c_best=None; y_best=None
    for _, fr in cand.iterrows():
        fipe_model_norm = fr["_model_norm"]
        c_toks = fr["_toks"]
        q_toks = v_toks
        inter = len(q_toks & c_toks)
        coverage = inter / len(q_toks) if q_toks else 0.0
        precision = inter / len(c_toks) if c_toks else 0.0
        if coverage < 0.05: continue
        f1 = (2*precision*coverage/(precision+coverage)) if (precision+coverage) else 0.0
        jacc = (len(q_toks & c_toks)/len(q_toks | c_toks)) if (q_toks or c_toks) else 0.0
        base = SequenceMatcher(None, version_norm, fipe_model_norm).ratio()
        score = 0.55*f1 + 0.20*jacc + 0.25*base
        c_engine = fr["_engine"]
        if v_engine:
            score += 0.05 if c_engine == v_engine else (-0.10 if c_engine is not None else 0.0)
        if "gp" in v_toks and "gp" in c_toks: score += 0.03
        score -= min(0.12, 0.02 * len(c_toks - q_toks))  # Small penalty for extra unmatched tokens
        if score > s_best:
            s_best = score; m_best = fr["Modelo"]; c_best = fr["CodigoFipe"]; y_best = int(fr["AnoModelo"]) if pd.notna(fr["AnoModelo"]) else None
    if s_best >= threshold and m_best and c_best and y_best is not None:
        return (m_best, c_best, y_best, float(round(s_best,4)), "matched")
    return (None, None, None, float(max(s_best,0.0)), "unmatched")

print('Canonical normalization & matching helpers loaded.')

## 5. Loading Raw Localiza CSV
Reads the latest Localiza snapshot selected earlier. Expected columns (case-insensitive):
- brand, model, version, model_year, price (others are kept if present)

If column names differ slightly the pipeline still tries to adapt. If the file is missing this cell stops execution with a clear message.

In [ ]:
if localiza_csv is None:
    raise SystemExit('Localiza CSV not found. Place parsed CSV in raw/localiza/.')
loc_df = pd.read_csv(localiza_csv, sep=';')
log.info('Localiza rows: %d columns: %d', len(loc_df), loc_df.shape[1])
required_loc_cols = {'brand','model','version','model_year','price'}
missing = required_loc_cols - set(c.lower() for c in loc_df.columns)
if missing:
    log.warning('Localiza missing columns (may be fine if naming differs): %s', missing)
loc_df.head()

## 6. Loading Raw Movida CSV
Same logic as Localiza. If no Movida file is found we continue with an empty Movida DataFrame (the rest of the pipeline still works – Movida outputs will just be empty).

In [ ]:
if movida_csv is None:
    log.warning('Movida CSV not found; continuing with Localiza only.')
    mov_df = pd.DataFrame(columns=['brand','model','version','model_year','price'])
else:
    mov_df = pd.read_csv(movida_csv, sep=';')
    log.info('Movida rows: %d columns: %d', len(mov_df), mov_df.shape[1])
mov_df.head()

## 7. Data Harmonization (Column Mapping & Typing)
We standardize both vendors to a common schema and data types:
- Force text fields to lowercase trimmed strings
- Convert prices to integer Reais
- Convert model_year to integer (dropping invalid years)
- Add a snapshot_date (today) if not present

Result: both DataFrames are comparable downstream.

In [ ]:
def _standardize(df: pd.DataFrame, vendor: str) -> pd.DataFrame:
    """Ensure required columns exist and enforce consistent types."""
    rename_map = {c.lower(): c for c in df.columns}
    def col(name):
        return next((c for k,c in rename_map.items() if k == name), None)
    # Create missing mandatory text columns
    for required in ['brand','model','version']:
        if col(required) is None:
            df[required] = ''
    # Create defaults if price/year absent
    if col('price') is None:
        df['price'] = pd.NA
    if col('model_year') is None:
        df['model_year'] = pd.NA
    if 'snapshot_date' not in df.columns:
        df['snapshot_date'] = today_iso()
    # Standardize casing
    for c in ['brand','model','version']:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()
    # Normalize price & year
    df['price'] = df['price'].apply(clean_price_to_int).astype('Int64') if 'price' in df.columns else pd.NA
    df['model_year'] = pd.to_numeric(df['model_year'], errors='coerce').astype('Int64')
    return df

loc_df = _standardize(loc_df, 'localiza')
mov_df = _standardize(mov_df, 'movida')
loc_df.head()

## 8. Augment Localiza (Derived Version Fields)
Adds normalized columns used for matching:
- _brand_norm (after aliasing)
- _model_norm (generic normalization)
- _version_norm (detailed normalization of the full version string)
- _engine (engine size like 1.0, 1.3, 2.0 if detected)

These are helper features for similarity scoring vs FIPE.

In [ ]:
for df in [loc_df]:
    if df.empty: continue
    df['_brand_norm'] = df['brand'].map(norm_brand)
    df['_model_norm'] = df['model'].map(generic_norm_text)
    df['_version_norm'] = df['version'].map(norm_text)
    df['_engine'] = df['_version_norm'].map(extract_engine)
loc_df[['brand','model','version','_version_norm']].head()

## 9. Load FIPE Models (fipe_models.csv)
Loads the master list of FIPE (Marca, Modelo, CodigoFipe, AnoModelo). This file must already be exported and placed under `data/`.

If the file is missing we stop, because no matching can happen without it.

In [ ]:
if fipe_models_csv is None:
    raise SystemExit('fipe_models.csv not found in data/. Download or copy it before proceeding.')
# Flexible separator autodetect
try:
    fipe_models = pd.read_csv(fipe_models_csv, sep=None, engine='python', encoding='utf-8-sig')
except Exception:
    fipe_models = pd.read_csv(fipe_models_csv, sep=';', encoding='utf-8-sig')
log.info('FIPE models rows: %d', len(fipe_models))
fipe_models.head()

## 10. Prepare FIPE Models (Normalization & Tokenization)
We clean & normalize FIPE models so they are comparable with vendor versions:
- Lowercase & strip spaces
- Apply same normalization to brand/model
- Tokenize each model into `_toks` (a set of unique words)
- Extract engine displacement for extra scoring tie‑breakers
- Remove duplicates (same code, model, year)

Output: DataFrame `fm` used for version matching.

In [ ]:
fipe_models.columns = [c.replace('\ufeff','').strip() for c in fipe_models.columns]
colmap = {c.lower(): c for c in fipe_models.columns}
# Required columns
req_map = {k: colmap.get(k) for k in ['marca','modelo','codigofipe','anomodelo']}
missing = [k for k,v in req_map.items() if v is None]
if missing:
    raise SystemExit(f'Missing required FIPE model columns: {missing}')
fm = fipe_models.rename(columns={req_map['marca']:'Marca', req_map['modelo']:'Modelo', req_map['codigofipe']:'CodigoFipe', req_map['anomodelo']:'AnoModelo'})
fm['Marca'] = fm['Marca'].astype(str).str.lower().str.strip()
fm['Modelo'] = fm['Modelo'].astype(str).str.lower().str.strip()
fm['CodigoFipe'] = fm['CodigoFipe'].astype(str).str.strip()
fm['AnoModelo'] = pd.to_numeric(fm['AnoModelo'], errors='coerce').astype('Int64')
fm = fm[fm['CodigoFipe'].ne('') & fm['Modelo'].ne('') & fm['AnoModelo'].notna()].copy()
fm['_brand_norm'] = fm['Marca'].map(norm_brand)
fm['_model_norm'] = fm['Modelo'].map(norm_text)
fm['_toks'] = fm['_model_norm'].str.split().apply(set)
fm['_engine'] = fm['_model_norm'].map(extract_engine)
fm = fm.drop_duplicates(subset=['CodigoFipe','Modelo','AnoModelo']).reset_index(drop=True)
fm.head()

## 11. Load / Initialize Version Match Cache
We keep a historical CSV (`data/localiza_version_match.csv`) so that once a (brand_norm, model_norm, version_norm, model_year) is matched to a FIPE code we do **not** need to recompute it every time.

Columns tracked:
- fipe_code + fipe_model + score
- match_source (method used)
- first_seen / last_seen (aging & churn analysis)

If the file does not exist we start with an empty cache.

In [ ]:
match_cols = ['brand_norm','model_norm','version_norm','model_year','fipe_brand','fipe_model','fipe_code','score','match_source','first_seen','last_seen']
# Load version match cache
if VERSION_MATCH_TABLE.exists():
    cache_df = pd.read_csv(VERSION_MATCH_TABLE, sep=';')
    for c in match_cols:
        if c not in cache_df.columns: cache_df[c] = pd.NA
    cache_df = cache_df[match_cols]
else:
    cache_df = pd.DataFrame(columns=match_cols)
cache_df.head()

## 12. Match Localiza Versions to FIPE Codes
For each new (brand_norm, model_norm, version_norm, model_year) key not already in the cache:
1. Filter FIPE candidates to same brand and year.
2. Keep candidates where FIPE model name contains the normalized model or all its tokens.
3. Score each candidate using token overlap & text similarity.
4. Pick the best candidate and record its FIPE code + score.

The threshold controls which matches are considered good enough later.

Result: `new_matches` DataFrame (may be empty if nothing new).

In [ ]:
THRESHOLD = 0.0  # Using 0.0 to avoid versions with no matches

# Identify new keys (those not already in the cache)
loc_df['model_year'] = pd.to_numeric(loc_df['model_year'], errors='coerce').astype('Int64')
key_cols = ['_brand_norm','_model_norm','_version_norm','model_year']
existing_keys = set(tuple(r) for r in cache_df[['brand_norm','model_norm','version_norm','model_year']].itertuples(index=False, name=None))
new_keys_df = (loc_df[key_cols].drop_duplicates()
               .rename(columns={'_brand_norm':'brand_norm','_model_norm':'model_norm','_version_norm':'version_norm'}))
new_keys_df['model_year'] = pd.to_numeric(new_keys_df['model_year'], errors='coerce').astype('Int64')
new_keys_df = new_keys_df[~new_keys_df.apply(tuple, axis=1).isin(existing_keys)]
log.info('New version keys to match: %d', len(new_keys_df))

rows = []
today_iso = today_iso()
for _, r in new_keys_df.iterrows():
    brand_n = r['brand_norm']; model_n = r['model_norm']; version_n = r['version_norm']; year = r['model_year']
    # Basic validation: need model tokens + year
    if pd.isna(year) or not model_n:
        rows.append({**r, 'fipe_brand': None, 'fipe_model': None, 'fipe_code': None, 'score': 0.0, 'match_source':'contains','first_seen':today_iso,'last_seen':today_iso})
        continue
    # Narrow FIPE candidates by brand & year
    cand = fm[(fm['_brand_norm']==brand_n) & (fm['AnoModelo']==int(year))]
    if cand.empty:
        rows.append({**r, 'fipe_brand': None, 'fipe_model': None, 'fipe_code': None, 'score': 0.0, 'match_source':'contains','first_seen':today_iso,'last_seen':today_iso})
        continue
    model_tok_set = set(model_n.split())
    # Require all model tokens to appear (or substring containment)
    cand = cand[cand['_model_norm'].apply(lambda m: model_n in m or model_tok_set.issubset(set(m.split())))]
    if cand.empty:
        rows.append({**r, 'fipe_brand': None, 'fipe_model': None, 'fipe_code': None, 'score': 0.0, 'match_source':'contains','first_seen':today_iso,'last_seen':today_iso})
        continue
    # Score remaining candidates
    v_toks = set(version_n.split()) if version_n else set()
    best_score=-1.0; best=None
    for _, fr in cand.iterrows():
        f_toks = fr['_toks']
        # Compute token intersection
        inter = len(v_toks & f_toks)
        # Compute coverage and precision
        coverage = inter/len(v_toks) if v_toks else 0.0
        precision = inter/len(f_toks) if f_toks else 0.0
        # Compute F1 score
        f1 = (2*precision*coverage/(precision+coverage)) if (precision+coverage) else 0.0
        # Compute Jaccard similarity
        jacc = (len(v_toks & f_toks)/len(v_toks | f_toks)) if (v_toks or f_toks) else 0.0
        # Compute sequence similarity
        seq = SequenceMatcher(None, version_n, fr['_model_norm']).ratio() if version_n else 0.0
        # Compute overall score
        score = 0.5*seq + 0.3*f1 + 0.2*jacc
        if score > best_score:
            best_score = score; best = fr
    if best is not None:
        rows.append({**r, 'fipe_brand': best['Marca'], 'fipe_model': best['Modelo'], 'fipe_code': best['CodigoFipe'], 'score': round(float(best_score),4), 'match_source':'contains','first_seen':today_iso,'last_seen':today_iso})

new_matches = pd.DataFrame(rows, columns=match_cols)
new_matches.head()

## 13. Persist Updated Match Cache & Matched Localiza Dataset
Steps:
1. Append any new matches to the cache.
2. Refresh `last_seen` for keys present today.
3. Save cache back to disk (so next time we reuse it).
4. Merge matches into Localiza rows producing: `localiza_with_fipe_match_YYYYMMDD.csv`.
5. Flag `match_accepted` when score >= threshold.

You can inspect the head below to sanity check a few rows.

In [ ]:
if not new_matches.empty:
    cache_df = pd.concat([cache_df, new_matches], ignore_index=True)
# Update last_seen for keys present today
present_keys = set(tuple(r) for r in loc_df[key_cols].drop_duplicates().rename(columns={'_brand_norm':'brand_norm','_model_norm':'model_norm','_version_norm':'version_norm'}).itertuples(index=False, name=None))
mask = cache_df[['brand_norm','model_norm','version_norm','model_year']].apply(tuple, axis=1).isin(present_keys)
cache_df.loc[mask,'last_seen'] = today_iso
cache_df = cache_df.sort_values(['brand_norm','model_norm','version_norm','model_year','last_seen']).drop_duplicates(subset=['brand_norm','model_norm','version_norm','model_year'], keep='last')
ensure_dir(VERSION_MATCH_TABLE.parent)
cache_df.to_csv(VERSION_MATCH_TABLE, index=False, sep=';')
log.info('Saved version match cache: %d rows', len(cache_df))

# Merge back into Localiza dataset
loc_matched = loc_df.merge(cache_df.rename(columns={'brand_norm':'_brand_norm','model_norm':'_model_norm','version_norm':'_version_norm'}), on=['_brand_norm','_model_norm','_version_norm','model_year'], how='left')
loc_matched.rename(columns={'score':'match_score'}, inplace=True)
loc_matched['match_score'] = pd.to_numeric(loc_matched['match_score'], errors='coerce').fillna(0.0)
loc_matched['match_accepted'] = (loc_matched['match_score'] >= THRESHOLD).astype(int)
match_out = MATCH_DIR / f"localiza_with_fipe_match_{ymd_compact()}.csv"
loc_matched.to_csv(match_out, index=False, sep=';')
log.info('Saved matched Localiza CSV: %s', match_out)
loc_matched[['brand','model','version','fipe_code','match_score','match_accepted']].head()

## 14. Extract (fipe_code, model_year) Tuples from Localiza & Movida
Collect unique pairs actually observed in vendor inventories. These drive:
- Which FIPE price rows we keep in time series (filter to what vendors sell)
- Audit / coverage monitoring

In [ ]:
def collect_tuples(loc: pd.DataFrame, mov: pd.DataFrame):
    out = set()
    if not loc.empty and 'fipe_code' in loc.columns:
        for _, r in loc[['fipe_code','model_year']].dropna().iterrows():
            code = str(r['fipe_code']).strip();
            try: yr = int(r['model_year'])
            except: continue
            if code:
                out.add((code, yr))
    if not mov.empty and 'fipe_code' in mov.columns:
        for _, r in mov[['fipe_code','model_year']].dropna().iterrows():
            code = str(r['fipe_code']).strip();
            try: yr = int(r['model_year'])
            except: continue
            if code:
                out.add((code, yr))
    return out

tuples_set = collect_tuples(loc_matched, mov_df)
log.info('Unique tuples collected: %d', len(tuples_set))
list(sorted(list(tuples_set))[:10])

## 15. Generate Tuples Audit DataFrame & Export
Creates a small table counting how many rows contributed each (fipe_code, model_year) per vendor.
Use this to see which models dominate stock. Saved to `data/tuples/` with today’s date.

In [ ]:
ldf = loc_matched[['fipe_code','model_year']].dropna().copy(); ldf['fipe_code']=ldf['fipe_code'].astype(str).str.strip()
mdf = mov_df[['fipe_code','model_year']].dropna().copy() if ('fipe_code' in mov_df.columns) else pd.DataFrame(columns=['fipe_code','model_year'])
if not mdf.empty:
    mdf['fipe_code']=mdf['fipe_code'].astype(str).str.strip()

lcnt = (ldf.groupby(['fipe_code','model_year']).size().rename('localiza_count').reset_index()) if not ldf.empty else pd.DataFrame(columns=['fipe_code','model_year','localiza_count'])
cnt2 = (mdf.groupby(['fipe_code','model_year']).size().rename('movida_count').reset_index()) if not mdf.empty else pd.DataFrame(columns=['fipe_code','model_year','movida_count'])
audit = pd.merge(lcnt, cnt2, on=['fipe_code','model_year'], how='outer')
for c in ['localiza_count','movida_count']:
    if c not in audit.columns: audit[c]=0
audit[['localiza_count','movida_count']] = audit[['localiza_count','movida_count']].fillna(0).astype(int)
audit['total_count'] = audit['localiza_count'] + audit['movida_count']
audit['localiza_present'] = audit['localiza_count'] > 0
audit['movida_present'] = audit['movida_count'] > 0
TUPLES_DIR.mkdir(parents=True, exist_ok=True)
audit_out = TUPLES_DIR / f'fipe_tuples_{ymd_compact()}.csv'
audit.to_csv(audit_out, index=False, sep=';')
log.info('Tuples audit saved (canonical schema): %s (rows=%d)', audit_out, len(audit))
audit.head()

## 16. Load Existing FIPE Dump
A richer FIPE dump with pricing (`ValorNum`, `MesReferencia`). If missing we still finish the pipeline, but price enrichment & time series become empty.

The file pattern: `data/fipe/fipe_dump_YYYYMMDD.csv` (we pick the newest).

In [ ]:
if fipe_dump_csv is None:
    log.warning('No FIPE dump CSV found (data/fipe/fipe_dump_*.csv). Skip pricing merge steps if absent.')
    fipe_dump = pd.DataFrame()
else:
    fipe_dump = pd.read_csv(fipe_dump_csv)
    # Convert price column to numeric if present (drops formatting issues)
    if 'ValorNum' in fipe_dump.columns:
        fipe_dump['ValorNum'] = pd.to_numeric(fipe_dump['ValorNum'], errors='coerce')
log.info('FIPE dump rows: %d', len(fipe_dump))
fipe_dump.head()

## 17. Build Localiza Vendor Table (Merge FIPE Pricing)
Joins Localiza rows with FIPE price for the same (fipe_code, model_year, snapshot_year, snapshot_month) so the price aligns with the snapshot month.

Additional rules:
- fipe_version is a deterministic mapping of fipe_code only (independent of model_year).
- SiglaCombustivel from FIPE dump is merged by fipe_code; if SiglaCombustivel in {E, H} we force type = EV.

Implementation details:
- If `MesReferencia` exists in FIPE dump we parse and shift (publish month -> market month = month-1) and match on (year, month).
- If `MesReferencia` is absent we fall back to code + model_year only (coarser price match).
- `premium_vs_fipe_price` = (offer_price – fipe_price) / fipe_price.

If no FIPE price data available the new columns will be blank (NA).

In [ ]:
# Canonical vendor table builder with row-count preservation (schema aligned with fleet.py build_vendor_table)
# Rules:
# - fipe_version derives ONLY from fipe_code (stable mapping from FIPE dump; year/month independent)
# - fipe_price chosen by (fipe_code, model_year, snapshot_year, snapshot_month) when MesReferencia exists, else by (fipe_code, model_year)
# - type normalization enhanced: merge SiglaCombustivel from FIPE dump by fipe_code. If SiglaCombustivel in {E,H} => type 'EV'
# Output columns: snapshot_date,snapshot_year,snapshot_month,type,brand,model,
# fipe_version,fipe_code,manufacture_year,model_year,offer_price,fipe_price,
# premium_vs_fipe_price,fipe_code_model_year,model_model_year

def build_vendor_table_offline(df: pd.DataFrame, fipe_dump: pd.DataFrame, vendor: str) -> pd.DataFrame:
    cols_schema = [
        'snapshot_date','snapshot_year','snapshot_month','type','brand','model','fipe_version','fipe_code',
        'manufacture_year','model_year','offer_price','fipe_price','premium_vs_fipe_price',
        'fipe_code_model_year','model_model_year'
    ]
    if df.empty:
        return pd.DataFrame(columns=cols_schema)
    work = df.copy()
    orig_len = len(work)
    work['_orig_row_id'] = range(orig_len)
    # Ensure required columns exist
    for c in ['snapshot_date','type','brand','model','manufacture_year','model_year','price','fipe_code']:
        if c not in work.columns:
            work[c] = pd.NA

    # Snapshot date decomposition
    work['snapshot_date'] = pd.to_datetime(work['snapshot_date'], errors='coerce')
    work['snapshot_year'] = work['snapshot_date'].dt.year
    work['snapshot_month'] = work['snapshot_date'].dt.month
    work['snapshot_date'] = work['snapshot_date'].dt.date.astype(str)

    # Build stable fipe_code -> fipe_version mapping (deterministic)
    fipe_version_map = {}
    if not fipe_dump.empty and {'CodigoFipe','Modelo'}.issubset(fipe_dump.columns):
        fv = (fipe_dump[['CodigoFipe','Modelo']]
                .dropna(subset=['CodigoFipe','Modelo'])
                .rename(columns={'CodigoFipe':'fipe_code','Modelo':'fipe_version'}))
        fv['len'] = fv['fipe_version'].str.len()
        fv = (fv.sort_values(['fipe_code','len','fipe_version'])
                .drop_duplicates(subset=['fipe_code']))
        fipe_version_map = dict(zip(fv['fipe_code'].astype(str), fv['fipe_version']))
    # Assign fipe_version purely from mapping
    work['fipe_version'] = work['fipe_code'].astype(str).map(fipe_version_map)

    # Helper to parse MesReferencia
    def _parse_mes_label(label: str):
        pt = {'janeiro':1,'fevereiro':2,'março':3,'marco':3,'abril':4,'maio':5,'junho':6,'julho':7,'agosto':8,'setembro':9,'outubro':10,'novembro':11,'dezembro':12}
        s = (label or '').lower().replace('\xa0',' ').strip()
        s = re.sub(r'\s+',' ', s)
        s_norm = re.sub(r'[/-]',' ', s)
        m = re.search(r'(janeiro|fevereiro|março|marco|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\s+(?:de\s+)?(\d{4})', s_norm)
        if m: return int(m.group(2)), pt[m.group(1)]
        m2 = re.search(r'(1[0-2]|0?[1-9])\s+(\d{4})', s_norm)
        if m2: return int(m2.group(2)), int(m2.group(1))
        return 1900,1

    # Merge FIPE price
    if not fipe_dump.empty and {'CodigoFipe','AnoModelo'}.issubset(fipe_dump.columns):
        fipe_sub = fipe_dump.copy()
        has_mes = 'MesReferencia' in fipe_sub.columns
        if has_mes:
            fipe_sub[['reference_year','reference_month']] = fipe_sub['MesReferencia'].map(_parse_mes_label).to_list()
            def _shift(y,m): return (y-1,12) if m==1 else (y,m-1)
            fipe_sub[['reference_year','reference_month']] = fipe_sub.apply(lambda r: _shift(int(r['reference_year']), int(r['reference_month'])), axis=1, result_type='expand')
        fipe_sub = fipe_sub.rename(columns={'CodigoFipe':'fipe_code','AnoModelo':'model_year','ValorNum':'fipe_price'})
        fipe_sub['fipe_price'] = pd.to_numeric(fipe_sub.get('fipe_price'), errors='coerce')
        base_cols = ['fipe_code','model_year','fipe_price'] + (['reference_year','reference_month'] if has_mes else [])
        fipe_sub = fipe_sub[[c for c in base_cols if c in fipe_sub.columns]].dropna(subset=['fipe_code','model_year'])
        if has_mes:
            fipe_sub = (fipe_sub
                        .sort_values(['fipe_code','model_year','reference_year','reference_month'])
                        .drop_duplicates(subset=['fipe_code','model_year','reference_year','reference_month'], keep='last'))
            work = work.merge(
                fipe_sub,
                left_on=['fipe_code','model_year','snapshot_year','snapshot_month'],
                right_on=['fipe_code','model_year','reference_year','reference_month'],
                how='left'
            )
            work.drop(columns=[c for c in ['reference_year','reference_month'] if c in work.columns], inplace=True)
        else:
            work = work.merge(fipe_sub, on=['fipe_code','model_year'], how='left')
        if len(work) != orig_len:
            work = (work.sort_values(['_orig_row_id'])
                        .groupby('_orig_row_id', as_index=False)
                        .first())
    else:
        if 'fipe_price' not in work.columns: work['fipe_price'] = pd.NA

    # Merge SiglaCombustivel by fipe_code (most frequent value per code)
    if not fipe_dump.empty and {'CodigoFipe','SiglaCombustivel'}.issubset(fipe_dump.columns):
        sig = (fipe_dump[['CodigoFipe','SiglaCombustivel']]
                 .dropna(subset=['CodigoFipe','SiglaCombustivel'])
                 .rename(columns={'CodigoFipe':'fipe_code'}))
        # choose most frequent per code
        sig = (sig.groupby('fipe_code')['SiglaCombustivel']
                 .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0])
                 .reset_index())
        work = work.merge(sig, on='fipe_code', how='left')

    # Normalize type incorporating SiglaCombustivel; E or H => EV
    base_type = work.apply(lambda r: normalize_type(r.get('type'), r.get('SiglaCombustivel')), axis=1)
    work['type'] = np.where(work['SiglaCombustivel'].isin(['E','H']), 'EV', base_type)

    work['offer_price'] = work.get('price')
    work['fipe_price'] = pd.to_numeric(work.get('fipe_price'), errors='coerce')
    work['premium_vs_fipe_price'] = np.where(work['fipe_price'].gt(0), (work['offer_price'] - work['fipe_price'])/work['fipe_price'], pd.NA)
    work['fipe_code_model_year'] = work['fipe_code'].astype(str) + '_' + work['model_year'].astype(str)
    work['model_model_year'] = work['model'].astype(str) + '_' + work['model_year'].astype(str)

    for c in cols_schema:
        if c not in work.columns:
            work[c] = pd.NA

    vendor_tbl = work[cols_schema].copy()
    if len(vendor_tbl) != orig_len:
        log.warning('%s vendor table row count mismatch (orig=%s, out=%s)', vendor, orig_len, len(vendor_tbl))

    # Diagnostics: confirm uniqueness of mapping
    if not vendor_tbl['fipe_version'].isna().all():
        inconsistent = vendor_tbl.groupby('fipe_code')['fipe_version'].nunique().gt(1)
        if inconsistent.any():
            log.warning('Inconsistent fipe_version mapping for some fipe_code.')
    return vendor_tbl

loc_vendor = build_vendor_table_offline(loc_matched, fipe_dump, 'localiza')

# Harmonize Onix Sedan Plus naming in Localiza vendor table
mask = (
    loc_vendor['fipe_version'].fillna('').str.lower().str.startswith('onix sedan plus') &
    loc_vendor['model'].fillna('').str.lower().eq('onix')
)
changed = mask.sum()
loc_vendor.loc[mask, 'fipe_version'] = 'onix plus'
print('Adjusted fipe_version to "onix plus" for rows:', changed)

loc_vendor.head()

## 18. Build Movida Vendor Table (Merge FIPE Pricing)
Same enrichment for Movida rows.

In [ ]:
mov_vendor = build_vendor_table_offline(mov_df, fipe_dump, 'movida')
# Validation: fipe_version uniqueness per fipe_code across both vendors
combined = pd.concat([loc_vendor[['fipe_code','fipe_version']], mov_vendor[['fipe_code','fipe_version']]])
uniques = combined.groupby('fipe_code')['fipe_version'].nunique()
violations = uniques[uniques>1]
print('Unique fipe_code values:', combined['fipe_code'].nunique())
print('fipe_code with >1 fipe_version (should be 0):', len(violations))
if len(violations):
    print('Violating codes:', violations.index.tolist()[:20])

mov_vendor.loc[mov_vendor['model'].str.lower()=='commander','type'] = 'suv'
mov_vendor.loc[mov_vendor['fipe_code'].str.lower()=='600005','fipe_code'] = '022189-9'
mov_vendor.head()

## 19. Build Consolidated FIPE Time Series Table
From the FIPE dump we build a monthly time series only for (fipe_code, model_year) pairs present in vendor stock.

Important details:
- FIPE month label refers to *publication month*; we shift one month back so price becomes valid for the actual market usage window.
- We compute `m_m_price_change` (percentage change vs previous month).

If no FIPE dump is present this table will be empty.

In [ ]:
def parse_mes_label(label: str):
    """Parse Portuguese month label (e.g. 'Julho de 2025') into (year, month)."""
    pt = {'janeiro':1,'fevereiro':2,'março':3,'marco':3,'abril':4,'maio':5,'junho':6,'julho':7,'agosto':8,'setembro':9,'outubro':10,'novembro':11,'dezembro':12}
    s = (label or '').lower().replace('\xa0',' ').strip()
    s = re.sub(r'\s+',' ', s)
    s_norm = re.sub(r'[/-]',' ', s)
    m = re.search(r'(janeiro|fevereiro|março|marco|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\s+(?:de\s+)?(\d{4})', s_norm)
    if m:
        return int(m.group(2)), pt[m.group(1)]
    m2 = re.search(r'(1[0-2]|0?[1-9])\s+(\d{4})', s_norm)
    if m2:
        return int(m2.group(2)), int(m2.group(1))
    return 1900,1  # Fallback improbable sentinel

if fipe_dump.empty:
    fipe_series = pd.DataFrame(columns=['reference_year','reference_month','brand','model','type','fipe_version','fipe_code','model_year','fipe_price','m_m_price_change'])
else:
    fdf = fipe_dump.copy()
    if 'MesReferencia' not in fdf.columns:
        log.warning('Missing MesReferencia in FIPE dump; cannot build time series')
        fipe_series = pd.DataFrame(columns=['reference_year','reference_month','brand','model','type','fipe_version','fipe_code','model_year','fipe_price','m_m_price_change'])
    else:
        fdf['reference_year'], fdf['reference_month'] = zip(*fdf['MesReferencia'].map(parse_mes_label))
        # Shift back one month (publish M -> market M-1)
        def _shift(y,m): return (y-1,12) if m==1 else (y, m-1)
        fdf['reference_year'], fdf['reference_month'] = zip(*fdf.apply(lambda r: _shift(int(r['reference_year']), int(r['reference_month'])), axis=1))
        fdf = fdf.rename(columns={'CodigoFipe':'fipe_code','AnoModelo':'model_year','ValorNum':'fipe_price','Marca':'brand','Modelo':'fipe_version'})
        # Map model presence from vendor tables
        model_map = pd.concat([
            loc_vendor[['fipe_code','model_year','model']],
            mov_vendor[['fipe_code','model_year','model']]
        ], ignore_index=True).dropna().drop_duplicates(subset=['fipe_code','model_year'])
        fdf = fdf.merge(model_map, on=['fipe_code','model_year'], how='left')
        # Keep only if model present in vendor fleet
        fdf = fdf[fdf['model'].notna()].copy()
        # Merge type preference (loc > mov) using already unified vendor tables
        try:
            type_map = pd.concat([
                loc_vendor[['fipe_code','model_year','type']].assign(_pref=0),
                mov_vendor[['fipe_code','model_year','type']].assign(_pref=1)
            ], ignore_index=True).dropna(subset=['type'])
            type_map = type_map.sort_values(['fipe_code','model_year','_pref']).drop_duplicates(subset=['fipe_code','model_year'])[['fipe_code','model_year','type']]
            fdf = fdf.merge(type_map, on=['fipe_code','model_year'], how='left')
        except Exception as e:
            log.warning('Type merge skipped: %s', e)
        fdf = fdf.sort_values(['fipe_code','model_year','reference_year','reference_month'])
        fdf = fdf.drop_duplicates(subset=['fipe_code','model_year','reference_year','reference_month'], keep='last')
        fdf['m_m_price_change'] = fdf.groupby(['fipe_code','model_year'])['fipe_price'].pct_change()
        fipe_series = fdf[['reference_year','reference_month','brand','model','type','fipe_version','fipe_code','model_year','fipe_price','m_m_price_change']]
fipe_series.head()

## 20. Compute Price Premium & Month-over-Month Changes
Nothing to run here: metrics are already produced in earlier steps (`premium_vs_fipe_price` in Sections 17–18, `m_m_price_change` in Section 19). Section kept for narrative completeness.

## 21. Finalize & Export All Output Tables
Writes dated CSVs so historical runs are preserved. You can safely re-run the entire notebook any day; new files will have a new date stamp.

Next steps after export:
- Update tables in "Car Price Index" file

In [ ]:
STAMP = ymd_compact()
ensure_dir(TABLES_DIR)
loc_vendor_out = TABLES_DIR / 'localiza' / f'localiza_table_{STAMP}.csv'
mov_vendor_out = TABLES_DIR / 'movida' / f'movida_table_{STAMP}.csv'
fipe_series_out = TABLES_DIR / 'fipe' / f'fipe_table_{STAMP}.csv'

# Row count preservation checks
raw_loc_len = len(loc_matched) if 'loc_matched' in globals() else len(loc_df)
raw_mov_len = len(mov_df)
if len(loc_vendor) != raw_loc_len:
    log.warning('Localiza vendor table rows (%s) differ from raw matched rows (%s)', len(loc_vendor), raw_loc_len)
if len(mov_vendor) != raw_mov_len:
    log.warning('Movida vendor table rows (%s) differ from raw rows (%s)', len(mov_vendor), raw_mov_len)

loc_vendor.to_csv(loc_vendor_out, index=False, sep=';')
mov_vendor.to_csv(mov_vendor_out, index=False, sep=';')
if not fipe_series.empty:
    fipe_series.to_csv(fipe_series_out, index=False, sep=';')
log.info('Exported canonical vendor tables and FIPE series (loc=%s rows, mov=%s rows).', len(loc_vendor), len(mov_vendor))
loc_vendor.head(), mov_vendor.head(), fipe_series.head()

---
### Quick Troubleshooting
| Symptom | Likely Cause | Fix |
|--------|--------------|-----|
| FileNotFound error for Localiza | No CSV in `raw/localiza/` | Put a file there and re-run from Section 2 |
| Zero new matches | Cache already covers all version keys | Normal; proceed |
| Many low scores (< threshold) | New trims / abbreviations | Add mapping inside normalization (Section 4) |
| Empty FIPE series table | No FIPE dump file found | Add a `fipe_dump_*.csv` to `data/fipe/` |
| premium_vs_fipe_price all blank | No price column or no FIPE price | Check raw file or add FIPE dump |

If stuck, copy the cell output and ask copilot.

## 22. Exploratory Analysis & Visualizations
Quick, optional data exploration for business users. Just run the cells below after exports.

If a plot cell shows a warning about a missing library (e.g. Plotly), you can still see static Matplotlib charts.

Skip any cell if underlying data is empty. Each cell is defensive and will tell you if it had nothing to show.

In [ ]:
# Visualization setup (uses matplotlib; tries plotly if installed)
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
try:
    import plotly.express as px
    _HAS_PLOTLY = True
except Exception:
    _HAS_PLOTLY = False

# Helper to show top N rows with safe fallback
def show_top(df, n=5):
    return df.head(n) if not df.empty else pd.DataFrame()

print('Plotly available:' , _HAS_PLOTLY)

### 22.1 Stock Composition by Body Type
Shows distribution of normalized type (if derivable) or approximated via version text tokens. Useful to see fleet mix.

In [ ]:
# Stock composition using canonical vendor table (loc_vendor)
# 'type' already normalized; fallback quick inference only if missing.
import re

def infer_type_from_model_row(r):
    s = (r.get('fipe_version') or r.get('model') or '').lower()
    if any(k in s for k in ['suv','compass','tracker','t cross','t-cross','renegade','hr-v','kicks','nivus']): return 'suv'
    if any(k in s for k in ['hatch','onix','gol','fox','hb20 ','argo','mobi','kwid']): return 'hatch'
    if any(k in s for k in ['sedan','virtus','voyage','corolla','cruze','civic','onix sedan','hb20s']): return 'sedan'
    if any(k in s for k in ['strada','s10','hilux','oro ch','oroch','ranger','amarok','l200','frontier']): return 'pickup'
    return 'other'

if 'loc_vendor' not in globals() or loc_vendor.empty:
    print('No Localiza vendor table available for stock composition.')
else:
    stock = loc_vendor.copy()
    if 'type' not in stock.columns or stock['type'].isna().all():
        stock['type'] = stock.apply(infer_type_from_model_row, axis=1)
    ct = stock['type'].fillna('unknown').value_counts().rename_axis('type').reset_index(name='count')
    if ct.empty:
        print('No data to plot.')
    else:
        if _HAS_PLOTLY:
            fig = px.bar(ct, x='type', y='count', title='Stock Composition (Localiza)', text='count')
            fig.update_traces(textposition='outside')
            fig.show()
        else:
            ax = ct.plot(kind='bar', x='type', y='count', legend=False, title='Stock Composition (Localiza)')
            for p in ax.patches:
                ax.annotate(int(p.get_height()), (p.get_x() + p.get_width()/2, p.get_height()), ha='center', va='bottom', fontsize=9)
            plt.show()
    ct.head()

### 22.2 Price Premium Distribution
Shows how Localiza asking prices compare vs FIPE reference (only matched rows with FIPE price).

In [ ]:
prem = loc_vendor.dropna(subset=['premium_vs_fipe_price','fipe_price'])
if prem.empty:
    print('No matched rows with FIPE price to analyze premium.')
else:
    prem['premium_pct'] = prem['premium_vs_fipe_price'] * 100
    if _HAS_PLOTLY:
        fig = px.histogram(prem, x='premium_pct', nbins=40, title='Price Premium vs FIPE (%)', marginal='box')
        fig.add_vline(x=0, line_dash='dash', line_color='black')
        fig.show()
    else:
        ax = prem['premium_pct'].plot(kind='hist', bins=40, title='Price Premium vs FIPE (%)')
        ax.axvline(0, color='black', linestyle='--')
        plt.xlabel('% Premium')
        plt.show()
    # Display sample rows (vendor schema has no raw version; show fipe_version instead)
    prem[['brand','model','fipe_version','premium_pct']].head()

### 22.3 Top Models by Stock & Premium
Table of models with highest presence and their median premium vs FIPE.

In [ ]:
top_base = loc_vendor.copy() if 'loc_vendor' in globals() else pd.DataFrame()
needed = {'brand','model','fipe_code','offer_price'}
if top_base.empty or not needed.issubset(top_base.columns):
    print('Not enough data for top models table.')
else:
    # Keep only rows with an offer price
    top_base = top_base[top_base['offer_price'].notna()]
    if top_base.empty:
        print('No non-null offer_price values available for aggregation.')
    else:
        # Compute aggregations; median skips NaN by default
        grp = (top_base.groupby(['brand','model'])
               .agg(stock=('fipe_code','count'),
                    avg_offer_price=('offer_price', lambda s: float(np.nanmean(s)) if s.notna().any() else np.nan),
                    median_premium=('premium_vs_fipe_price', lambda s: float(np.nanmedian(s)) if s.notna().any() else np.nan))
               .reset_index())
        # Drop groups with no price info
        grp = grp[grp['avg_offer_price'].notna()]
        if grp.empty:
            print('All groups lacked valid offer_price after filtering.')
        else:
            grp = grp.sort_values('stock', ascending=False)
            display_cols = ['brand','model','stock','avg_offer_price','median_premium']
            grp['avg_offer_price'] = grp['avg_offer_price'].round(0).astype(int)
            print('Top 15 models by stock (filtered for valid prices):')
            display(grp.head(15)[display_cols])

### 22.4 Time Series: Example FIPE Price Trend
Pick a (fipe_code, model_year) with enough history to view month-over-month price evolution.

In [ ]:
if 'fipe_series' not in globals() or fipe_series.empty:
    print('No FIPE series data loaded.')
else:
    freq = (fipe_series.groupby(['fipe_code','model_year'])
            .size().rename('points').reset_index().sort_values('points', ascending=False))
    if freq.empty:
        print('No time series points to plot.')
    else:
        fc, my = freq.iloc[0][['fipe_code','model_year']]
        sel = fipe_series[(fipe_series['fipe_code']==fc) & (fipe_series['model_year']==my)].copy()
        sel['date'] = pd.to_datetime(sel['reference_year'].astype(int).astype(str) + '-' + sel['reference_month'].astype(int).astype(str) + '-01')
        sel = sel.sort_values('date')
        if _HAS_PLOTLY:
            fig = px.line(sel, x='date', y='fipe_price', title=f'FIPE Price Trend (code={fc}, year={my})')
            fig.update_traces(mode='lines+markers')
            fig.show()
        else:
            plt.figure(figsize=(6,3))
            plt.plot(sel['date'], sel['fipe_price'], marker='o')
            plt.title(f'FIPE Price Trend (code={fc}, year={my})')
            plt.xlabel('Date'); plt.ylabel('Price')
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        sel[['date','fipe_price','m_m_price_change']].head()

### 22.5 Match Score Quality Bands
Understand distribution of match scores to monitor normalization / threshold health.

In [ ]:
# Match score distribution now must read from cache_df or loc_matched (pre-vendor collapse) since vendor tables lack match_score.
score_source = 'loc_matched' if 'loc_matched' in globals() else None
if not score_source:
    print('loc_matched not available for score distribution.')
else:
    scores_df = loc_matched[['match_score']].dropna()
    if scores_df.empty:
        print('No match scores to display.')
    else:
        bins = [0,0.4,0.5,0.6,0.7,0.8,0.9,1.01]
        labels = ['<0.4','0.4-0.5','0.5-0.6','0.6-0.7','0.7-0.8','0.8-0.9','0.9+']
        scores_df['band'] = pd.cut(scores_df['match_score'], bins=bins, labels=labels, include_lowest=True, right=False)
        dist = scores_df['band'].value_counts().reindex(labels).fillna(0).reset_index()
        dist.columns = ['band','count']
        if _HAS_PLOTLY:
            fig = px.bar(dist, x='band', y='count', title='Match Score Distribution', text='count')
            fig.update_traces(textposition='outside')
            fig.add_vline(x=labels.index('0.6-0.7'), line_color='orange', line_dash='dash')
            fig.show()
        else:
            ax = dist.plot(kind='bar', x='band', y='count', legend=False, title='Match Score Distribution')
            for p in ax.patches:
                ax.annotate(int(p.get_height()), (p.get_x()+p.get_width()/2, p.get_height()), ha='center', va='bottom')
            plt.show()
        dist

### 22.6 Notes & Next Ideas
- Add side-by-side comparison for Movida vs Localiza premiums.
- Track how match score distribution shifts after updating normalization rules.
- Export these plots automatically to an `outputs/` folder for sharing.

End of notebook.